# 제주도 교통량 예측 문제
**날씨 및 유동인구에 따른 교통량 추이**

- 데이터 출처 : https://www.jejudatahub.net/data/view/data/753

In [29]:
import pandas as pd

```{attention} 데이터 설명
제주빅데이터센터 데이터를 활용하여 효율적인 교통량 측정을 위한 날씨/유동인구 활용 교통량 추이 데이터 제공
 - 거주인구, 근무인구, 방문인구는 해당 인구수의 일별 유동인구 수 합계
 - 유동인구는 01~24시 해당 시간 정각 측정 인구 (머문 시간(분)/60분)
 - 교통량 (대), 평균 속도 (km), 평균 소요 시간 (초), 평균 기온 (°C), 일일강수량 (mm), 평균 풍속(m/s)
```

```{admonition} Question 1
**데이터를 로드하고 각데이터의 상위 3개 값을 출력하라**
```

In [33]:
train_X = pd.read_csv("https://raw.githubusercontent.com/Datamanim/jeju/main/Jeju_trainX.csv",encoding='euc-kr')
train_y= pd.read_csv("https://raw.githubusercontent.com/Datamanim/jeju/main/Jeju_trainy.csv",encoding='euc-kr')
test_X= pd.read_csv("https://raw.githubusercontent.com/Datamanim/jeju/main/Jeju_testX.csv",encoding='euc-kr')
sub= pd.read_csv("https://raw.githubusercontent.com/Datamanim/jeju/main/subExample.csv",encoding='euc-kr')


In [35]:
display(train_X.head(3))
display(train_y.head(3))
display(test_X.head(3))
display(sub.head(3))

,id,일자,시도명,읍면동명,거주인구,근무인구,방문인구,총 유동인구,평균 속도,평균 소요 시간,평균 기온,일강수량,평균 풍속
0,0,2018-01-01,서귀포시,남원읍,291408.897,18744.131,219588.857,529741.884,53.000,40.571,2.35,0.0,3.325
1,1,2018-01-01,제주시,노형동,1215420.248,74964.858,445735.329,1736120.435,38.563,30.655,-2.30,0.0,1.100
2,2,2018-01-01,제주시,구좌읍,276220.023,17542.105,248104.122,541866.250,53.188,40.719,5.90,0.0,4.100


,id,교통량
0,0,76.200
1,1,1291.184
2,2,0.000


,id,일자,시도명,읍면동명,거주인구,근무인구,방문인구,총 유동인구,평균 속도,평균 소요 시간,평균 기온,일강수량,평균 풍속
0,22448,2018-01-01,제주시,도두동,32249.987,3418.266,102709.092,138377.345,39.556,29.167,5.0,0.0,2.5
1,22449,2018-01-01,제주시,외도동,213500.997,10341.172,112692.789,336534.958,32.900,30.900,5.0,0.0,2.5
2,22450,2018-01-01,제주시,이도2동,1212382.218,96920.834,541194.481,1850497.533,29.538,35.692,2.9,0.0,2.4


,id,교통량
0,22448,0
1,22449,0
2,22450,0


```{admonition} Question 2
**각데이터의 결측치, 데이터타입, 연속형변수의 분포를 파악하라**
```

info() , insull().sum(), describe() 함수 이용

```{admonition} Question 3
**LabelEncoder를 이용하여 trainset의 읍면동, 시도명을 fit하고 train_X,test_X를 각각 transform 하여 pre_train, pre_test 변수에 저장하고 상위 5개값을 출력하라**
```

In [39]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(train_X['읍면동명'] )

train_X['읍면동명']  = le.transform(train_X['읍면동명'])
test_X['읍면동명']  = le.transform(test_X['읍면동명'])

train_X['시도명'] = train_X['시도명'].map(lambda x: 1 if x=='서귀포시' else 0) 
test_X['시도명'] = test_X['시도명'].map(lambda x: 1 if x=='서귀포시' else 0) 

In [40]:
train_X[['시도명','읍면동명']].head(6)

,시도명,읍면동명
0,0,2
1,0,3
2,0,1
3,0,24
4,0,0
5,0,25


```{admonition} Question 4
**'id' 컬럼을 제거하라. '일자'를 년도,월,일,요일(숫자)로 각각 분리하여 컬럼을 형성하라. 그후 '일자'컬럼을 제거하라**
```

In [48]:
def datePreprocessing(df2):
    
    df =df2.copy()
    df['일자'] = pd.to_datetime(df['일자'])
    df['년도'] = df['일자'].dt.year
    df['월'] = df['일자'].dt.month
    df['일'] = df['일자'].dt.day
    df['weekend'] = df['일자'].dt.weekday    
    
    
    return df

pre_train = datePreprocessing(train_X).drop(['일자','id'],axis=1)
pre_test = datePreprocessing(test_X).drop(['일자','id'],axis=1)

In [49]:
pre_train.head(3)

,시도명,읍면동명,거주인구,근무인구,방문인구,총 유동인구,평균 속도,평균 소요 시간,평균 기온,일강수량,평균 풍속,년도,월,일,weekend
0,0,2,291408.897,18744.131,219588.857,529741.884,53.000,40.571,2.35,0.0,3.325,2018,1,1,0
1,0,3,1215420.248,74964.858,445735.329,1736120.435,38.563,30.655,-2.30,0.0,1.100,2018,1,1,0
2,0,1,276220.023,17542.105,248104.122,541866.250,53.188,40.719,5.90,0.0,4.100,2018,1,1,0


```{admonition} Question 5
**인구 관련 컬럼(거주인구,근무인구,방문인구,총 유동인구를 정규화하라. 정규화시 trainset에 대한 fit을 test셋에 적용하라.**
```

In [50]:
from sklearn.preprocessing import StandardScaler

sc =StandardScaler()

def pre2(df,sc =None):
    df2 = df.copy()
    if sc ==None:
        sc = StandardScaler()
        sc.fit(df2[['거주인구','근무인구','방문인구','총 유동인구']])
    df2[['거주인구','근무인구','방문인구','총 유동인구']] = sc.transform(df2[['거주인구','근무인구','방문인구','총 유동인구']])
    
    return df2,sc

pre_train , sc = pre2(pre_train)
pre_test , sc2 = pre2(pre_test,sc)

In [52]:
pre_train.head(3)

,시도명,읍면동명,거주인구,근무인구,방문인구,총 유동인구,평균 속도,평균 소요 시간,평균 기온,일강수량,평균 풍속,년도,월,일,weekend
0,0,2,-0.087699,-0.418008,0.166825,-0.042833,53.000,40.571,2.35,0.0,3.325,2018,1,1,0
1,0,3,3.011526,0.959452,1.775292,2.573146,38.563,30.655,-2.30,0.0,1.100,2018,1,1,0
2,0,1,-0.138644,-0.447459,0.369639,-0.016541,53.188,40.719,5.90,0.0,4.100,2018,1,1,0


```{admonition} Question 6
**적당한 회귀 모델을 선정하고, 학습 및 mean_squared_error 값을 통해 모델을 검증하라**
```

In [14]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(random_state=24)

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(pre_train, train_y['교통량'], test_size=0.15, random_state=42)
rf.fit(X_train,y_train)
pred = rf.predict(X_test)

from sklearn.metrics import mean_squared_error

mse = mean_squared_error(pred,y_test)

In [60]:
mse

7169.674733186257

```{admonition} Question 7
**모델을 통해 테스트셋의 예측값을 추출하고 제출파일을 생성 및 저장하라.**
```

In [62]:
sub['교통량'] =rf.predict(pre_test)
sub.to_csv('./submission.csv')

In [63]:
sub.head(3)

,id,교통량
0,22448,1369.73863
1,22449,1107.06700
2,22450,313.38008


```{tip} 
**저장이 잘됐다면 저장된 테스트 셋의 mse값을 아래의 코드를 통해 확인해볼수 있습니다.**
```

In [64]:
def FinalMseScore():
    import pandas as pd
    y_true = pd.read_csv("https://raw.githubusercontent.com/Datamanim/jeju/main/Jeju_testy.csv",encoding='euc-kr')
    sub = pd.read_csv('./submission.csv',index_col='id')
    pred = sub.iloc[:,-1].values
    from sklearn.metrics import mean_squared_error
    mse = mean_squared_error(pred,y_true['교통량'])    
    print('submission mse score : ',mse)
    return mse

In [65]:
final_mse = FinalMseScore()

submission mse score :  6797.38846471611
